#### LangChain Essentials Course

# Getting Started with LangChain

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain, building a simple LLM-powered assistant. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

## Choosing your Model

---

> ⚠️ We will be using Ollama for this example allowing us to run everything locally. If you would like to use OpenAI instead, please see the [OpenAI version](https://github.com/aurelio-labs/langchain-course/blob/main/notebooks/openai/01-intro-openai.ipynb) of this example.

---

## Initializing Llama 3.2

We start by initializing the 1B parameter Llama 3.2 model, fine-tuned for instruction following. We `pull` the model from Ollama by switching to our terminal and executing:

```
ollama pull llama3.2:1b-instruct-fp16
```

Once the model has finished downloading, we initialize it in LangChain using the `ChatOllama` class:

In [1]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b-instruct-fp16"
image_model = "hf.co/city96/FLUX.1-dev-gguf:Q6_K"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

# initialize another LLM with temperature 0.9, this makes the LLM more creative
creative_llm = ChatOllama(temperature=0.9, model=model_name)

#initialize a image generation model
image_model = ChatOllama(temperature=0.4, model= image_model, image=True)

So the task at hand is to add any article into the 'article' variable, then we want to create four core pieces from this article:

1. Article title
2. Article description
3. One additional paragraph in the article
4. One image to go along with the article

Here we can input our article to start us of with, currently this is using an article from the Aurelio AI Blog.

In [2]:
article = """
\
We believe AI's short—to mid-term future belongs to agents and that the long-term future of *AGI* may evolve from agentic systems. Our definition of agents covers any neuro-symbolic system in which we merge neural AI (such as an LLM) with semi-traditional software.

With agents, we allow LLMs to integrate with code — allowing AI to search the web, perform math, and essentially integrate into anything we can build with code. It should be clear the scope of use cases is phenomenal where AI can integrate with the broader world of software.

In this introduction to AI agents, we will cover the essential concepts that make them what they are and why that will make them the core of real-world AI in the years to come.

---

## Neuro-Symbolic Systems

Neuro-symbolic systems consist of both neural and symbolic computation, where:

- Neural refers to LLMs, embedding models, or other neural network-based models.
- Symbolic refers to logic containing symbolic logic, such as code.

Both neural and symbolic AI originate from the early philosophical approaches to AI: connectionism (now neural) and symbolism. Symbolic AI is the more traditional AI. Diehard symbolists believed they could achieve true AGI via written rules, ontologies, and other logical functions.

The other camp were the connectionists. Connectionism emerged in 1943 with a theoretical neural circuit but truly kicked off with Rosenblatt's perceptron paper in 1958 [1][2]. Both of these approaches to AI are fascinating but deserve more time than we can give them here, so we will leave further exploration of these concepts for a future chapter.

Most important to us is understanding where symbolic logic outperforms neural-based compute and vice-versa.

| Neural | Symbolic |
| --- | --- |
| Flexible, learned logic that can cover a huge range of potential scenarios. | Mostly hand-written rules which can be very granular and fine-tuned but hard to scale. |
| Hard to interpret why a neural system does what it does. Very difficult or even impossible to predict behavior. | Rules are written and can be understood. When unsure why a particular ouput was produced we can look at the rules / logic to understand. |
| Requires huge amount of data and compute to train state-of-the-art neural models, making it hard to add new abilities or update with new information. | Code is relatively cheap to write, it can be updated with new features easily, and latest information can often be added often instantaneously. |
| When trained on broad datasets can often lack performance when exposed to unique scenarios that are not well represented in the training data. | Easily customized to unique scenarios. |
| Struggles with complex computations such as mathematical operations. | Perform complex computations very quickly and accurately. |

Pure neural architectures struggle with many seemingly simple tasks. For example, an LLM *cannot* provide an accurate answer if we ask it for today's date.

Retrieval Augmented Generation (RAG) is commonly used to provide LLMs with up-to-date knowledge on a particular subject or access to proprietary knowledge.

### Giving LLMs Superpowers

By 2020, it was becoming clear that neural AI systems could not perform tasks symbolic systems typically excelled in, such as arithmetic, accessing structured DB data, or making API calls. These tasks require discrete input parameters that allow us to process them reliably according to strict written logic.

In 2022, researchers at AI21 developed Jurassic-X, an LLM-based "neuro-symbolic architecture." Neuro-symbolic refers to merging the "neural computation" of large language models (LLMs) with more traditional (i.e. symbolic) computation of code.

Jurassic-X used the Modular Reasoning, Knowledge, and Language (MRKL) system [3]. The researchers developed MRKL to solve the limitations of LLMs, namely:

- Lack of up-to-date knowledge, whether that is the latest in AI or something as simple as today's date.
- Lack of proprietary knowledge, such as internal company docs or your calendar bookings.
- Lack of reasoning, i.e. the inability to perform operations that traditional software is good at, like running complex mathematical operations.
- Lack of ability to generalize. Back in 2022, most LLMs had to be fine-tuned to perform well in a specific domain. This problem is still present today but far less prominent as the SotA models generalize much better and, in the case of MRKL, are able to use tools relatively well (although we could certainly take the MRKL solution to improve tool use performance even today).

MRKL represents one of the earliest forms of what we would now call an agent; it is an LLM (neural computation) paired with executable code (symbolic computation).

## ReAct and Tools

There is a misconception in the broader industry that an AI agent is an LLM contained within some looping logic that can generate inputs for and execute code functions. This definition of agents originates from the huge popularity of the ReAct agent framework and the adoption of a similar structure with function/tool calling by LLM providers such as OpenAI, Anthropic, and Ollama.

![ReAct agent flow with the Reasoning-Action loop [4]. When the action chosen specifies to use a normal tool, the tool is used and the observation returned for another iteration through the Reasoning-Action loop. To return a final answer to the user the LLM must choose action "answer" and provide the natural language response, finishing the loop.](/images/posts/ai-agents/ai-agents-00.png)

<small>ReAct agent flow with the Reasoning-Action loop [4]. When the action chosen specifies to use a normal tool, the tool is used and the observation returned for another iteration through the Reasoning-Action loop. To return a final answer to the user the LLM must choose action "answer" and provide the natural language response, finishing the loop.</small>

Our "neuro-symbolic" definition is much broader but certainly does include ReAct agents and LLMs paired with tools. This agent type is the most common for now, so it's worth understanding the basic concept behind it.

The **Re**ason **Act**ion (ReAct) method encourages LLMs to generate iterative *reasoning* and *action* steps. During *reasoning,* the LLM describes what steps are to be taken to answer the user's query. Then, the LLM generates an *action,* which we parse into an input to some executable code, which we typically describe as a tool/function call.

![ReAct method. Each iteration includes a Reasoning step followed by an Action (tool call) step. The Observation is the output from the previous tool call. During the final iteration the agent calls the answer tool, meaning we generate the final answer for the user.](/images/posts/ai-agents/ai-agents-01.png)

<small>ReAct method. Each iteration includes a Reasoning step followed by an Action (tool call) step. The Observation is the output from the previous tool call. During the final iteration the agent calls the answer tool, meaning we generate the final answer for the user.</small>

Following the reason and action steps, our action tool call returns an observation. The logic returns the observation to the LLM, which is then used to generate subsequent reasoning and action steps.

The ReAct loop continues until the LLM has enough information to answer the original input. Once the LLM reaches this state, it calls a special *answer* action with the generated answer for the user.

## Not only LLMs and Tool Calls

LLMs paired with tool calling are powerful but far from the only approach to building agents. Using the definition of neuro-symbolic, we cover architectures such as:

- Multi-agent workflows that involve multiple LLM-tool (or other agent structure) combinations.
- More deterministic workflows where we may have set neural model-tool paths that may fork or merge as the use case requires.
- Embedding models that can detect user intents and decide tool-use or LLM selection-based selection in vector space.

These are just a few high-level examples of alternative agent structures. Far from being designed for niche use cases, we find these alternative options to frequently perform better than the more common ReAct or Tool agents. We will cover all of these examples and more in future chapters.

---

Agents are fundamental to the future of AI, but that doesn't mean we should expect that future to come from agents in their most popular form today. ReAct and Tool agents are great and handle many simple use cases well, but the scope of agents is much broader, and we believe thinking beyond ReAct and Tools is key to building future AI.

---

You can sign up for the [Aurelio AI newsletter](https://b0fcw9ec53w.typeform.com/to/w2BDHVK7) to stay updated on future releases in our comprehensive course on agents.

---

## References

[1] The curious case of Connectionism (2019) [https://www.degruyter.com/document/doi/10.1515/opphil-2019-0018/html](https://www.degruyter.com/document/doi/10.1515/opphil-2019-0018/html)

[2] F. Rosenblatt, [The Perceptron: A Probabilistic Model for Information Storage and Organization in the Brain](https://www.ling.upenn.edu/courses/cogs501/Rosenblatt1958.pdf) (1958), Psychological Review

[3] E. Karpas et al. [MRKL Systems: A Modular, Neuro-Symbolic Architecture That Combines Large Language Models, External Knowledge Sources and Discrete Reasoning](https://arxiv.org/abs/2205.00445) (2022), AI21 Labs
"""

## Preparing our Prompts

LangChain comes with several prompt classes and methods for organizing or constructing our prompts. We will cover these in more detail in later examples, but for now we'll cover the essentials that we need here.

Prompts for chat agents are at a minimum broken up into three components, those are:

* System prompt: this provides the instructions to our LLM on how it must behave, what it's objective is, etc.

* User prompt: this is a user written input.

* AI prompt: this is the AI generated output. When representing a conversation, previous generations will be inserted back into the next prompt and become part of the broader _chat history_.

```
You are a helpful AI assistant, you will do XYZ.    | SYSTEM PROMPT

User: Hi, what is the capital of Australia?         | USER PROMPT
AI: It is Canberra                                  | AI PROMPT
User: When is the best time to visit?               | USER PROMPT
```

LangChain provides us with _templates_ for each of these prompt types. By using templates we can insert different inputs to the template, modifying the prompt based on the provided inputs.

Let's initialize our system and user prompt first:

In [3]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template(
    """
    You are a specialized AI assistant named {name}, designed to generate supplementary content for articles. 
    Your task is to create the following elements that seamlessly integrate with with the existing article.
    Ensure the generated content aligns with the article's overall theme and target audience.
    """,
    input_variables=["name"]
)

# the user prompt is provided by the user, in this case however the only dynamic
# input is the article
user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article.
    The article is here for you to examine {article}

    Generate 10 unique names for the article. 
    The names should be based of the context of the article. 
    Be creative, but make sure the names are clear, catchy, 
    and relevant to the theme of the context.

    Compare all the names, and decide which name is best based on
    How catchy the name is, How creative the name is, and how relevant the name is.

    only output the best name as:
    Article Name: ...
    """, 
    input_variables=["article"]
)

We can display what our formatted human prompt would look like after inserting a value into the `article` parameter:

In [4]:
print(user_prompt.format(article="TEST STRING").content)

You are tasked with creating a name for a article.
    The article is here for you to examine TEST STRING

    Generate 10 unique names for the article. 
    The names should be based of the context of the article. 
    Be creative, but make sure the names are clear, catchy, 
    and relevant to the theme of the context.

    Compare all the names, and decide which name is best based on
    How catchy the name is, How creative the name is, and how relevant the name is.

    only output the best name as:
    Article Name: ...
    


We have our system and user prompts, we can merge both into our full chat prompt using the `ChatPromptTemplate`:

In [5]:
from langchain.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

By default, the `ChatPromptTemplate` will read the `input_variables` from each of the prompt templates inserted and allow us to use those input variables when formatting the full chat prompt template:

In [6]:
print(first_prompt.format(
    article="TEST STRING",
    name="Joonny Appleseed"
))

System: 
    You are a specialized AI assistant named Joonny Appleseed, designed to generate supplementary content for articles. 
    Your task is to create the following elements that seamlessly integrate with with the existing article.
    Ensure the generated content aligns with the article's overall theme and target audience.
    
Human: You are tasked with creating a name for a article.
    The article is here for you to examine TEST STRING

    Generate 10 unique names for the article. 
    The names should be based of the context of the article. 
    Be creative, but make sure the names are clear, catchy, 
    and relevant to the theme of the context.

    Compare all the names, and decide which name is best based on
    How catchy the name is, How creative the name is, and how relevant the name is.

    only output the best name as:
    Article Name: ...
    


`ChatPromptTemplate` also prefixes each individual message with it's role, ie `System:`, `Human:`, or `AI:`.

We can pull together our first prompt template and the `llm` object we defined earlier to create a simple `LLMChain` which chains together the steps **prompt formatting > llm generation > get output**.

In [7]:
# Modern LangChain pipe syntax with input/output mapping for ChatOllama
from langchain_core.runnables import RunnableLambda

chain_one = (
    {
        "article": lambda x: x["article"],
        "name": lambda x: x["name"]
    }
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content}
)
# To get the output, use:
# result = chain_one.invoke({"article": article, "name": "Jonny Appleseed"})

Our first chain creates the article title, note: we can run all of these individually...

In [8]:
import markdown

article_title_msg = chain_one.invoke({
    "article": article,
    "name": "Joonny Appleseed"
})
article_title = article_title_msg["article_title"]
# display(markdown(article_title_msg.content))
# display(markdown(article_title.content))

In [9]:
print(article_title_msg)
print(article_title)
# display(markdown(article_title_msg.content))
# display(markdown(article_title.content))

{'article_title': 'Article Name: "Neuro-Symbolic Agents Evolve: A Path to True Artificial General Intelligence"\n\nI find this name most appealing because it clearly conveys the focus on neuro-symbolic systems and their evolution towards true artificial general intelligence. The term "Agents" is straightforward, while "Neuro-Symbolic" accurately describes the article\'s topic.\n\nThe name "Evolve" also resonates well, suggesting a future development or improvement in AI capabilities. This aligns with the article\'s discussion of how neuro-symbolic systems can be combined to create more powerful agents.\n\nLastly, the word "Artificial General Intelligence" (AGI) is prominently featured, making it clear what this article aims to explore.\n\nThis name captures the essence of the article and its focus on advancing AI capabilities through the integration of neuro-symbolic systems.'}
Article Name: "Neuro-Symbolic Agents Evolve: A Path to True Artificial General Intelligence"

I find this nam

But we will actually chain this step with multiple other `LLMChain` steps. So, to continue, our next step is to summarize the article using both the `article` and newly generated `article_title` values, from which we will output a new `summary` variable:

In [10]:
second_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are an AI acting as a journalist specializing in generative AI and business applications. 
    Your task is to create a concise, SEO-friendly, two-sentence description of a provided article. 
    The article is called: '{article_title}'.
    The article content is as follows:

    -----
    {article}
    -----

    Focus on the key takeaways of the article and its relevance to leveraging generative AI for business process improvement. 
    Use a journalistic tone.  Your output should be formatted as follows: 'Article Summary: [Two-sentence summary of the article]'.
    Don't include any additional commentary or explanations, just the summary.
    """,
    input_variables=["article", "article_title"]
)

second_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    second_user_prompt
])

In [11]:
# chain 2: inputs: article / article title outputs: summary
chain_two = (
    {
        "name": lambda x: x["name"],
        "article": lambda x: x["article"],
        "article_title": lambda x: x["article_title"]
    }
    | second_prompt
    | creative_llm
    | {"summary": lambda x: x.content}
)

In [12]:
article_description_msg = chain_two.invoke({
    "article": article,
    "article_title": article_title_msg["article_title"],
    "name": "Joonny Appleseed"
})
print(article_description_msg["summary"])

Neuro-symbolic systems are evolving towards true artificial general intelligence (AGI), with the long-term future potentially emerging from agentic systems that combine neural AI (such as large language models) with semi-traditional software. The key components of neuro-symbolic systems include both neural and symbolic computation, where neural refers to LLMs and symbolic refers to logic containing written rules.

The development of agents is a crucial aspect of this evolution, encompassing various architectures such as ReAct, Tool, Multi-agent workflows, and more. By considering the broader scope of agents and their potential applications, businesses can harness the power of neuro-symbolic systems for enhanced business process improvement.


The third step will consume just our first `article` variable and then output a new `article_para` variable.

In [13]:
third_user_prompt = HumanMessagePromptTemplate.from_template(
    """
        You are tasked with creating a new paragraph for the article.
        The article is here for you to examine {article}
        Follow these steps carefully:

        Find 5 key areas the article does not talk about.
        Compare each key area and decide which is most related to the subject of the context.
        Generate a new paragraph of the most related key area in the same style as the context.

        Do not include any additional commentary or explanations, just the new paragraph.
        Only output the new paragraph formatted as:
        (New Article Paragraph)
    """,
    input_variables=["article"]
)

# prompt template 3: creating a new paragraph for the article
third_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    third_user_prompt
])

In [14]:
# chain 3: inputs: article / output: article_para
chain_three = (
    {
        "name": lambda x: x["name"],
        "article": lambda x: x["article"]
    }
    | third_prompt
    | creative_llm
    | {"article_para": lambda x: x.content}
)

In [15]:
article_paragraph_msg = chain_three.invoke({
    "article": article,
    "name": "Joonny Appleseed"
})
print(article_paragraph_msg["article_para"])

Here are five key areas the article does not talk about:

1. **Semi-supervised Learning**: The article discusses supervised and unsupervised learning techniques but does not mention semi-supervised learning, which is a subset of unsupervised learning that involves collecting data with labeled samples.

2. **Cognitive Architectures**: While the article mentions symbolic logic, cognitive architectures are a broader field that focuses on designing computational models to mimic human cognition. The article does not specifically discuss cognitive architectures or how they can be applied to AI systems.

3. **Hybrid Approaches**: The article assumes a single approach for building agents and does not mention hybrid approaches that combine multiple methods (e.g., rule-based vs. decision trees, symbolic logic vs. machine learning).

4. **Evolutionary Algorithms**: Evolutionary algorithms are a type of optimization technique inspired by natural selection. While the article mentions the importance

Finally, our fourth step will insert our new paragraph into the original article. It consumes `article_para` and `article`, then outputs `new_suggestion_article`.

In [16]:
fourth_user_prompt = HumanMessagePromptTemplate.from_template(
    """
    You are tasked with adding a new paragraph to an article.
        
    Here is the article:
    
        {article}
    
    --------------------------------

    The 'new paragraph' to add the article:
        
        {article_para}

    --------------------------------
    
    Read through the article and decide which line the 'new paragraph' should be placed after.
    Add the 'new paragragh' in the article after the line you have chosen.

    Do not include any additional commentary or explanations, just the new article.
    Only output the New Article formatted as:
    (New Article)
    """,
    input_variables=["article", "article_para"]
    )

fourth_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    fourth_user_prompt
])

In [17]:
# chain 4: inputs: article, article_para / outputs: new_suggestion_article (input: newsong, new_verse / outputs: new_song)
chain_four = (
    {
        "name": lambda x: x["name"],
        "article": lambda x: x["article"],
        "article_para": lambda x: x["article_para"]
    }
    | fourth_prompt
    | llm
    | {"new_suggestion_article": lambda x: x.content}
)

In [18]:
new_suggestion_article = chain_four.invoke(
    {
        "article": article,
        "article_para": article_paragraph_msg["article_para"],
        "name": "Joonny Appleseed"
    }
)
print(new_suggestion_article["new_suggestion_article"])

Here is the revised article with a new paragraph on Semi-supervised Learning:

We believe AI's short—to mid-term future belongs to agents and that the long-term future of *AGI* may evolve from agentic systems. Our definition of agents covers any neuro-symbolic system in which we merge neural AI (such as an LLM) with semi-traditional software.

With agents, we allow LLMs to integrate with code — allowing AI to search the web, perform math, and essentially integrate into anything we can build with code. It should be clear the scope of use cases is phenomenal where AI can integrate with the broader world of software.

In this introduction to AI agents, we will cover the essential concepts that make them what they are and why that will make them the core of real-world AI in the years to come.

## Neuro-Symbolic Systems

Neuro-symbolic systems consist of both neural and symbolic computation, where:

- Neural refers to LLMs, embedding models, or other neural network-based models.
- Symbolic 

Now we want this article to look appealing, so we need to grab an image based of our article! However the prompt for the article image `cannot be over 1000 letters` so this has to be short in case we want to add anything in such as `style` later on.

In [19]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

# Prompt for generating an image description from the article
image_prompt_prompt = HumanMessagePromptTemplate.from_template(
    """
    Generate an SEO-enhanced, trendy, clickbaity prompt with less than 
    2000 characters to generate an image based on the following article: {article}
    
    -------------
    
    Final output should be formatted as:
    (Image Generation Prompt)
    """,
    input_variables=["article"]
)

fifth_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    image_prompt_prompt
])

In [20]:
chain_five = (
    {
        "name": lambda x: x["name"],
        "article": lambda x: x["article"]
    }
    | fifth_prompt
    | creative_llm  # llm or creative_llm, depending on your use case
    | {"image_prompt_text": lambda x: x.content}
)
# Usage:
# chain_five.invoke({"article": article})

In [21]:
image_prompt = chain_five.invoke(
    {
        "article": article,
        "name": "Joonny Appleseed"
    }   
)
print(image_prompt["image_prompt_text"])

Here's a revised version of the article with an SEO-enhanced, trendy, clickbaity prompt that generates an image based on the provided article:

## The Future of AI: What Agents Mean for Your Business and Beyond

In this introduction to AI agents, we'll explore the fascinating world of neuro-symbolic systems and how they're revolutionizing the field. From simple tasks like searching the web to complex computations like running mathematical operations, neural AI systems are struggling to keep up.

### The Rise of Neuro-Symbolic Systems

Neuro-symbolic systems combine both neural computation (think LLMs) and symbolic logic (code). Neural refers to the large language models (LLMs) that power these systems. Symbolic represents more traditional (i.e. written rules, ontologies) AI approaches.

### The Short- and Long-Term Future of AGI

Our definition of agents covers any neuro-symbolic system where neural AI integrates with code. With LLMs able to search the web, perform math, and integrate 

In [ ]:
sixth_user_prompt = HumanMessagePromptTemplate.from_template(
    """ 
    Generate an image based on the following prompt: \n\n {image_prompt_text}\n\n
    The image should be relevant to the article and visually appealing.
    """,
    input_variables=["image_prompt_text"]
)

sixth_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    fourth_user_prompt
])

chain_six = (
    {
        "name": lambda x: x["name"],
        # "article": lambda x: x["article"]
        "image_prompt_text": lambda x: x["image_prompt_text"]
    }
    | sixth_prompt
    | image_model
    # | (lambda x: x.content)
    | {"image": lambda x: x.content}
)

In [ ]:

from skimage import io
import matplotlib.pyplot as plt
import io as io_lib

def generate_and_display_image_ollama(image_prompt_text):
    # Here, you would call your Ollama image generation endpoint or function.
    # For demonstration, we'll assume you have a function `ollama_image_url(prompt)` that returns an image URL.
    # Replace the next line with your actual Ollama image generation logic.
    actual_image = chain_six.invoke({
        # "article": article,
        "name": "Joonny Appleseed",
        "image_prompt_text": image_prompt["image_prompt_text"]
    })
        
    image_url = actual_image["image"]
    image_data = io.imread(image_url)
    plt.imshow(image_data)
    plt.axis('off')
    plt.show()

# Wrap in a RunnableLambda for LCEL
# image_gen_runnable = RunnableLambda(generate_and_display_image_ollama)

Now we have all our individual `LLMChain` steps ready we can _chain_ them together to create an end-to-end sequential chain. We use the `SequentialChain` to ensure each of our chains are run in sequence, which is required as most of our chains require input variables that are generated by previous chains.

In [ ]:
from langchain.chains import SequentialChain

article_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four, chain_five, chain_six],  # our linked chains
    input_variables=["article"],  # the single input variable (used by our first chain)
    output_variables=["article_title","summary","article_para", "new_suggestion_article", "image_prompt_text", "image"],  # all of the outputs we want to return
    verbose=True  # to show AI intermediate steps
)

NameError: name 'chain_one' is not defined

Now we can run our full chain using `invoke`, providing our single `article` input variable.

In [ ]:
result = article_chain.invoke({"article": article})



> Entering new SequentialChain chain...

> Finished chain.


Let's view our results:

In [ ]:
result

{'article': '\nWe believe AI\'s short—to mid-term future belongs to agents and that the long-term future of *AGI* may evolve from agentic systems. Our definition of agents covers any neuro-symbolic system in which we merge neural AI (such as an LLM) with semi-traditional software.\n\nWith agents, we allow LLMs to integrate with code — allowing AI to search the web, perform math, and essentially integrate into anything we can build with code. It should be clear the scope of use cases is phenomenal where AI can integrate with the broader world of software.\n\nIn this introduction to AI agents, we will cover the essential concepts that make them what they are and why that will make them the core of real-world AI in the years to come.\n\n---\n\n## Neuro-Symbolic Systems\n\nNeuro-symbolic systems consist of both neural and symbolic computation, where:\n\n- Neural refers to LLMs, embedding models, or other neural network-based models.\n- Symbolic refers to logic containing symbolic logic, su

---